In [1]:
import pandas as pd

In [2]:
prec_hourly = pd.read_csv('data/MODIS/precipitation.csv')

In [3]:
prec_hourly.head()

,date,hourlyPrecipRate
0,20140301_0000,0.314555
1,20140301_0100,0.215825
2,20140301_0200,0.001990
3,20140301_0300,0.001698
4,20140301_0400,0.000853


In [4]:
dates = prec_hourly['date'].to_list()

Convert to date to datetime objects

In [5]:
new_dates = []
for date in dates:
    new_date = f'{date[0:4]}-{date[4:6]}-{date[6:8]} {date[9:11]}:00:00'
    new_dates.append(new_date)
prec_hourly['date'] = new_dates
prec_hourly['date']= pd.to_datetime(prec_hourly['date'])
prec_hourly = prec_hourly.set_index('date')

In [6]:
prec_hourly

,hourlyPrecipRate
date,
2014-03-01 00:00:00,0.314555
2014-03-01 01:00:00,0.215825
2014-03-01 02:00:00,0.001990
2014-03-01 03:00:00,0.001698
2014-03-01 04:00:00,0.000853
...,...
2017-12-31 19:00:00,0.000000
2017-12-31 20:00:00,0.000000
2017-12-31 21:00:00,0.000000


Resample to daily intervals

In [7]:
prec_daily = prec_hourly.resample('D').sum()

In [8]:
prec_daily

,hourlyPrecipRate
date,
2014-03-01,0.619397
2014-03-02,0.526817
2014-03-03,16.570071
2014-03-04,1.718126
2014-03-05,28.216512
...,...
2017-12-27,0.000000
2017-12-28,6.014218
2017-12-29,79.895810
